In [1]:
import pandas as pd
import numpy as np

products_df = pd.read_csv("../data/warehouse_products.csv")
products_df.head()

,date,product_id,warehouse_id,demand,inbound_units_weekly,actual_outbound_weekly,inventory_level_weekly,unmet_demand_weekly,stockout_flag_weekly,inbound_units_jit,actual_outbound_jit,inventory_level_jit,unmet_demand_jit,stockout_flag_jit
0,2024-01-01,P001,WH1,29,0,29,221,0,False,0,29,221,0,False
1,2024-01-01,P002,WH1,23,0,23,227,0,False,0,23,227,0,False
2,2024-01-01,P003,WH1,34,0,34,216,0,False,0,34,216,0,False
3,2024-01-01,P004,WH1,21,0,21,229,0,False,0,21,229,0,False
4,2024-01-01,P005,WH1,28,0,28,222,0,False,0,28,222,0,False


In [2]:
df = products_df[["date", "warehouse_id", "inbound_units", "actual_outbound", "inventory_level", "unmet_demand"]]

In [3]:
df.head()

,date,warehouse_id,inbound_units,actual_outbound,inventory_level,unmet_demand
0,2024-01-01,WH1,0,31,169,0
1,2024-01-01,WH1,0,26,174,0
2,2024-01-01,WH1,0,37,163,0
3,2024-01-01,WH1,0,37,163,0
4,2024-01-01,WH1,0,23,177,0


In [4]:
group_df = df.groupby(["date", "warehouse_id"])
group_df = group_df.agg({"inbound_units": "sum", "actual_outbound": "sum", "inventory_level": "sum", "unmet_demand": "sum"})
group_df = group_df.reset_index()

In [5]:
group_df = group_df.rename(columns={"actual_outbound": "orders_fulfilled", "unmet_demand": "missed_sales"})

In [6]:
van_capacity = 100
group_df["outbound_shipments"] = group_df["orders_fulfilled"] / van_capacity

In [7]:
group_df["outbound_shipments"] = group_df["outbound_shipments"].apply(np.ceil).astype("int")

In [8]:
warehouse_capacity = 2500

group_df["utilization"] = group_df["inventory_level"] / warehouse_capacity

In [9]:
truck_capacity = 1000

group_df["inbound_shipments"] = group_df["inbound_units"] / truck_capacity
group_df["inbound_shipments"] = group_df["inbound_shipments"].apply(np.ceil).astype("int")

In [10]:
def determine_staff_count(row):
    outbound_orders = row["orders_fulfilled"]
    inbound_shipments = row["inbound_shipments"]
    if outbound_orders < 100:
        return 2
    elif outbound_orders <= 150:
        return 3
    elif outbound_orders <= 200:
        return 4
    else:
        return 5

group_df["staff_count"] = group_df.apply(determine_staff_count, axis=1)

In [11]:
group_df["staff_count"] = group_df.apply(determine_staff_count, axis=1)

In [12]:
group_df.to_csv("../data/warehouse_daily.csv", index=False)